In [239]:
# importing necessary libraries
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.model_selection import cross_val_score

In [240]:
# exporting data
data = pd.read_csv('data.csv')
data.head()

,Cost Rank,Product Name,Product Life Cycle,FY20 Q4,FY21 Q1,FY21 Q2,FY21 Q3,FY21 Q4,FY22 Q1,FY22 Q2,FY22 Q3,FY22 Q4,FY23 Q1,FY23 Q2,FY23 Q3,FY23Q4 Demand,FYQ23 Q4 Marketing,FY23 Q4 Stats/ML
0,1,ROUTER 1,Sustaining,NaN,NaN,NaN,906.0,941.0,663.0,769.0,3565.0,1020,1670,38,2141,"1,716","1,716","1,948"
1,2,TRANSCIEVER,Sustaining,87173.0,102849.0,126375.0,128629.0,179125.0,138750.0,208760.0,116126.0,150803,82163,82408,67132,"85,000","141,285","97,995"
2,3,SWITCH 1,Sustaining,NaN,NaN,NaN,3651.0,10409.0,10575.0,17934.0,15104.0,15525,6938,9595,9231,"12,000","11,820","11,184"
3,4,ACCESS POINT 1,Sustaining,NaN,NaN,NaN,NaN,NaN,NaN,9576.0,46029.0,109452,54955,60478,62708,"62,771","79,113","65,310"
4,5,ACCESS POINT 2,Sustaining,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26769,33670,74853,80944,"76,192","113,704","93,200"


In [241]:
#removing unnecessary columns
data = data.drop(['Cost Rank','Product Life Cycle','FY23Q4 Demand','FYQ23 Q4 Marketing','FY23 Q4 Stats/ML'], axis = 1)
data.set_index('Product Name', inplace = True)
data.head()

,FY20 Q4,FY21 Q1,FY21 Q2,FY21 Q3,FY21 Q4,FY22 Q1,FY22 Q2,FY22 Q3,FY22 Q4,FY23 Q1,FY23 Q2,FY23 Q3
Product Name,,,,,,,,,,,,
ROUTER 1,NaN,NaN,NaN,906.0,941.0,663.0,769.0,3565.0,1020,1670,38,2141
TRANSCIEVER,87173.0,102849.0,126375.0,128629.0,179125.0,138750.0,208760.0,116126.0,150803,82163,82408,67132
SWITCH 1,NaN,NaN,NaN,3651.0,10409.0,10575.0,17934.0,15104.0,15525,6938,9595,9231
ACCESS POINT 1,NaN,NaN,NaN,NaN,NaN,NaN,9576.0,46029.0,109452,54955,60478,62708
ACCESS POINT 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26769,33670,74853,80944


In [242]:
data.shape

(20, 12)

In [243]:
transposed = data.transpose()
transposed.shape

(12, 20)

In [244]:
transposed

Product Name,ROUTER 1,TRANSCIEVER,SWITCH 1,ACCESS POINT 1,ACCESS POINT 2,SWITCH 2,SWITCH 3,POWER SUPPLY 1,SWITCH 4,SWITCH 5,SWITCH 6,ACCESS POINT 3,SUPERVISOR ENGINE,SWITCH 7,WIRELESS CONTROLLER,SWITCH 8,SWITCH 9,ACCESS POINT 4,SWITCH 10,POWER SUPPLY 2
FY20 Q4,NaN,87173.0,NaN,NaN,NaN,13432.0,28585.0,115709.0,NaN,4571.0,NaN,65048.0,NaN,6073.0,1208.0,4.0,6187.0,NaN,NaN,28279.0
FY21 Q1,NaN,102849.0,NaN,NaN,NaN,12463.0,19686.0,98337.0,NaN,4100.0,NaN,67810.0,NaN,5459.0,963.0,-4.0,5301.0,NaN,NaN,22704.0
FY21 Q2,NaN,126375.0,NaN,NaN,NaN,13452.0,44724.0,147681.0,NaN,4830.0,NaN,90928.0,NaN,6736.0,1698.0,36.0,7172.0,NaN,NaN,24098.0
FY21 Q3,906.0,128629.0,3651.0,NaN,NaN,14581.0,37781.0,185327.0,177.0,5472.0,NaN,148547.0,NaN,12213.0,1833.0,305.0,7582.0,NaN,NaN,31461.0
FY21 Q4,941.0,179125.0,10409.0,NaN,NaN,15831.0,31905.0,194170.0,270.0,7391.0,NaN,228945.0,NaN,13883.0,2423.0,1093.0,10964.0,NaN,NaN,47009.0
FY22 Q1,663.0,138750.0,10575.0,NaN,NaN,13864.0,37888.0,220386.0,850.0,7894.0,NaN,204184.0,NaN,27045.0,2602.0,2360.0,11501.0,NaN,NaN,35145.0
FY22 Q2,769.0,208760.0,17934.0,9576.0,NaN,15012.0,32689.0,204604.0,2717.0,7101.0,222.0,245453.0,15.0,15162.0,2341.0,4317.0,14980.0,NaN,NaN,45978.0
FY22 Q3,3565.0,116126.0,15104.0,46029.0,NaN,11349.0,30442.0,194538.0,3833.0,6943.0,1549.0,245803.0,108.0,13855.0,1830.0,4343.0,9158.0,NaN,NaN,34518.0
FY22 Q4,1020.0,150803.0,15525.0,109452.0,26769.0,10422.0,27028.0,185160.0,4279.0,6126.0,4619.0,136283.0,1972.0,12351.0,1567.0,4094.0,11601.0,13260.0,1227.0,40605.0
FY23 Q1,1670.0,82163.0,6938.0,54955.0,33670.0,8726.0,21246.0,147284.0,6089.0,4616.0,4764.0,76020.0,2427.0,8933.0,1210.0,2756.0,3788.0,21667.0,24186.0,26925.0


In [245]:
transposed.index

Index(['FY20 Q4', 'FY21 Q1', 'FY21 Q2', 'FY21 Q3', 'FY21 Q4', 'FY22 Q1',
       'FY22 Q2', 'FY22 Q3', 'FY22 Q4', 'FY23 Q1', 'FY23 Q2', 'FY23 Q3'],
      dtype='object')

In [246]:
transposed = transposed.reset_index().rename(columns={'index': 'Quarters'})


In [247]:
# transposed.index

In [248]:
# transposed.columns

In [249]:
transposed

Product Name,Quarters,ROUTER 1,TRANSCIEVER,SWITCH 1,ACCESS POINT 1,ACCESS POINT 2,SWITCH 2,SWITCH 3,POWER SUPPLY 1,SWITCH 4,...,SWITCH 6,ACCESS POINT 3,SUPERVISOR ENGINE,SWITCH 7,WIRELESS CONTROLLER,SWITCH 8,SWITCH 9,ACCESS POINT 4,SWITCH 10,POWER SUPPLY 2
0,FY20 Q4,NaN,87173.0,NaN,NaN,NaN,13432.0,28585.0,115709.0,NaN,...,NaN,65048.0,NaN,6073.0,1208.0,4.0,6187.0,NaN,NaN,28279.0
1,FY21 Q1,NaN,102849.0,NaN,NaN,NaN,12463.0,19686.0,98337.0,NaN,...,NaN,67810.0,NaN,5459.0,963.0,-4.0,5301.0,NaN,NaN,22704.0
2,FY21 Q2,NaN,126375.0,NaN,NaN,NaN,13452.0,44724.0,147681.0,NaN,...,NaN,90928.0,NaN,6736.0,1698.0,36.0,7172.0,NaN,NaN,24098.0
3,FY21 Q3,906.0,128629.0,3651.0,NaN,NaN,14581.0,37781.0,185327.0,177.0,...,NaN,148547.0,NaN,12213.0,1833.0,305.0,7582.0,NaN,NaN,31461.0
4,FY21 Q4,941.0,179125.0,10409.0,NaN,NaN,15831.0,31905.0,194170.0,270.0,...,NaN,228945.0,NaN,13883.0,2423.0,1093.0,10964.0,NaN,NaN,47009.0
5,FY22 Q1,663.0,138750.0,10575.0,NaN,NaN,13864.0,37888.0,220386.0,850.0,...,NaN,204184.0,NaN,27045.0,2602.0,2360.0,11501.0,NaN,NaN,35145.0
6,FY22 Q2,769.0,208760.0,17934.0,9576.0,NaN,15012.0,32689.0,204604.0,2717.0,...,222.0,245453.0,15.0,15162.0,2341.0,4317.0,14980.0,NaN,NaN,45978.0
7,FY22 Q3,3565.0,116126.0,15104.0,46029.0,NaN,11349.0,30442.0,194538.0,3833.0,...,1549.0,245803.0,108.0,13855.0,1830.0,4343.0,9158.0,NaN,NaN,34518.0
8,FY22 Q4,1020.0,150803.0,15525.0,109452.0,26769.0,10422.0,27028.0,185160.0,4279.0,...,4619.0,136283.0,1972.0,12351.0,1567.0,4094.0,11601.0,13260.0,1227.0,40605.0
9,FY23 Q1,1670.0,82163.0,6938.0,54955.0,33670.0,8726.0,21246.0,147284.0,6089.0,...,4764.0,76020.0,2427.0,8933.0,1210.0,2756.0,3788.0,21667.0,24186.0,26925.0


In [250]:
transposed['Quarters'] = ['2020 Q4','2021 Q1','2021 Q2','2021 Q3','2021 Q4','2022 Q1','2022 Q2','2022 Q3','2022 Q4','2023 Q1','2023 Q2','2023 Q3']

In [251]:
transposed

Product Name,Quarters,ROUTER 1,TRANSCIEVER,SWITCH 1,ACCESS POINT 1,ACCESS POINT 2,SWITCH 2,SWITCH 3,POWER SUPPLY 1,SWITCH 4,...,SWITCH 6,ACCESS POINT 3,SUPERVISOR ENGINE,SWITCH 7,WIRELESS CONTROLLER,SWITCH 8,SWITCH 9,ACCESS POINT 4,SWITCH 10,POWER SUPPLY 2
0,2020 Q4,NaN,87173.0,NaN,NaN,NaN,13432.0,28585.0,115709.0,NaN,...,NaN,65048.0,NaN,6073.0,1208.0,4.0,6187.0,NaN,NaN,28279.0
1,2021 Q1,NaN,102849.0,NaN,NaN,NaN,12463.0,19686.0,98337.0,NaN,...,NaN,67810.0,NaN,5459.0,963.0,-4.0,5301.0,NaN,NaN,22704.0
2,2021 Q2,NaN,126375.0,NaN,NaN,NaN,13452.0,44724.0,147681.0,NaN,...,NaN,90928.0,NaN,6736.0,1698.0,36.0,7172.0,NaN,NaN,24098.0
3,2021 Q3,906.0,128629.0,3651.0,NaN,NaN,14581.0,37781.0,185327.0,177.0,...,NaN,148547.0,NaN,12213.0,1833.0,305.0,7582.0,NaN,NaN,31461.0
4,2021 Q4,941.0,179125.0,10409.0,NaN,NaN,15831.0,31905.0,194170.0,270.0,...,NaN,228945.0,NaN,13883.0,2423.0,1093.0,10964.0,NaN,NaN,47009.0
5,2022 Q1,663.0,138750.0,10575.0,NaN,NaN,13864.0,37888.0,220386.0,850.0,...,NaN,204184.0,NaN,27045.0,2602.0,2360.0,11501.0,NaN,NaN,35145.0
6,2022 Q2,769.0,208760.0,17934.0,9576.0,NaN,15012.0,32689.0,204604.0,2717.0,...,222.0,245453.0,15.0,15162.0,2341.0,4317.0,14980.0,NaN,NaN,45978.0
7,2022 Q3,3565.0,116126.0,15104.0,46029.0,NaN,11349.0,30442.0,194538.0,3833.0,...,1549.0,245803.0,108.0,13855.0,1830.0,4343.0,9158.0,NaN,NaN,34518.0
8,2022 Q4,1020.0,150803.0,15525.0,109452.0,26769.0,10422.0,27028.0,185160.0,4279.0,...,4619.0,136283.0,1972.0,12351.0,1567.0,4094.0,11601.0,13260.0,1227.0,40605.0
9,2023 Q1,1670.0,82163.0,6938.0,54955.0,33670.0,8726.0,21246.0,147284.0,6089.0,...,4764.0,76020.0,2427.0,8933.0,1210.0,2756.0,3788.0,21667.0,24186.0,26925.0


In [252]:
# Split the 'Quarters' column into year and quarter
transposed[['Year', 'Quarter']] = transposed['Quarters'].str.split(' ', expand=True)

# Map quarter strings to their numeric representations
quarter_map = {'Q1': 8, 'Q2': 11, 'Q3': 2, 'Q4': 5} 
transposed['Quarter'] = transposed['Quarter'].map(quarter_map)

# Convert to datetime
transposed['Date'] = pd.to_datetime(transposed['Year'] + '-' + transposed['Quarter'].astype(str), format='%Y-%m')

# Convert to period
transposed['Quarter'] = transposed['Date'].dt.to_period('Q')

In [253]:
# # Split the 'Quarters' column into year and quarter
# transposed[['Year', 'Quarter']] = transposed['Quarters'].str.split('Q', expand=True)
# # 
# # Map quarter strings to their numeric representations
# quarter_map = {'Q1': 8, 'Q2': 11, 'Q3': 2, 'Q4': 5}  # Adjusted mapping for fiscal year
# transposed['Quarter'] = transposed['Quarter'].map(quarter_map)

# # Adjust year for quarters in Q1 and Q2
# transposed.loc[transposed['Quarter'].isin([8, 11]), 'Year'] = transposed['Year'].astype(int) - 1

# # Convert to datetime
# transposed['Date'] = pd.to_datetime(transposed['Year'] + '-' + transposed['Quarter'].astype(str), format='%Y-%m')

# # Convert to period
# transposed['Quarter'] = transposed['Date'].dt.to_period('Q')

In [254]:
transposed

Product Name,Quarters,ROUTER 1,TRANSCIEVER,SWITCH 1,ACCESS POINT 1,ACCESS POINT 2,SWITCH 2,SWITCH 3,POWER SUPPLY 1,SWITCH 4,...,SWITCH 7,WIRELESS CONTROLLER,SWITCH 8,SWITCH 9,ACCESS POINT 4,SWITCH 10,POWER SUPPLY 2,Year,Quarter,Date
0,2020 Q4,NaN,87173.0,NaN,NaN,NaN,13432.0,28585.0,115709.0,NaN,...,6073.0,1208.0,4.0,6187.0,NaN,NaN,28279.0,2020,2020Q2,2020-05-01
1,2021 Q1,NaN,102849.0,NaN,NaN,NaN,12463.0,19686.0,98337.0,NaN,...,5459.0,963.0,-4.0,5301.0,NaN,NaN,22704.0,2021,2021Q3,2021-08-01
2,2021 Q2,NaN,126375.0,NaN,NaN,NaN,13452.0,44724.0,147681.0,NaN,...,6736.0,1698.0,36.0,7172.0,NaN,NaN,24098.0,2021,2021Q4,2021-11-01
3,2021 Q3,906.0,128629.0,3651.0,NaN,NaN,14581.0,37781.0,185327.0,177.0,...,12213.0,1833.0,305.0,7582.0,NaN,NaN,31461.0,2021,2021Q1,2021-02-01
4,2021 Q4,941.0,179125.0,10409.0,NaN,NaN,15831.0,31905.0,194170.0,270.0,...,13883.0,2423.0,1093.0,10964.0,NaN,NaN,47009.0,2021,2021Q2,2021-05-01
5,2022 Q1,663.0,138750.0,10575.0,NaN,NaN,13864.0,37888.0,220386.0,850.0,...,27045.0,2602.0,2360.0,11501.0,NaN,NaN,35145.0,2022,2022Q3,2022-08-01
6,2022 Q2,769.0,208760.0,17934.0,9576.0,NaN,15012.0,32689.0,204604.0,2717.0,...,15162.0,2341.0,4317.0,14980.0,NaN,NaN,45978.0,2022,2022Q4,2022-11-01
7,2022 Q3,3565.0,116126.0,15104.0,46029.0,NaN,11349.0,30442.0,194538.0,3833.0,...,13855.0,1830.0,4343.0,9158.0,NaN,NaN,34518.0,2022,2022Q1,2022-02-01
8,2022 Q4,1020.0,150803.0,15525.0,109452.0,26769.0,10422.0,27028.0,185160.0,4279.0,...,12351.0,1567.0,4094.0,11601.0,13260.0,1227.0,40605.0,2022,2022Q2,2022-05-01
9,2023 Q1,1670.0,82163.0,6938.0,54955.0,33670.0,8726.0,21246.0,147284.0,6089.0,...,8933.0,1210.0,2756.0,3788.0,21667.0,24186.0,26925.0,2023,2023Q3,2023-08-01


In [255]:
transposed.columns

Index(['Quarters', 'ROUTER 1', 'TRANSCIEVER', 'SWITCH 1', 'ACCESS POINT 1',
       'ACCESS POINT 2', 'SWITCH 2', 'SWITCH 3', 'POWER SUPPLY 1', 'SWITCH 4',
       'SWITCH 5', 'SWITCH 6', 'ACCESS POINT 3', 'SUPERVISOR ENGINE',
       'SWITCH 7', 'WIRELESS CONTROLLER', 'SWITCH 8', 'SWITCH 9',
       'ACCESS POINT 4', 'SWITCH 10', 'POWER SUPPLY 2', 'Year', 'Quarter',
       'Date'],
      dtype='object', name='Product Name')

In [256]:
transposed.drop(['Quarters', 'Quarter', 'Year'], axis=1, inplace=True)

In [257]:
transposed.columns

Index(['ROUTER 1', 'TRANSCIEVER', 'SWITCH 1', 'ACCESS POINT 1',
       'ACCESS POINT 2', 'SWITCH 2', 'SWITCH 3', 'POWER SUPPLY 1', 'SWITCH 4',
       'SWITCH 5', 'SWITCH 6', 'ACCESS POINT 3', 'SUPERVISOR ENGINE',
       'SWITCH 7', 'WIRELESS CONTROLLER', 'SWITCH 8', 'SWITCH 9',
       'ACCESS POINT 4', 'SWITCH 10', 'POWER SUPPLY 2', 'Date'],
      dtype='object', name='Product Name')

In [258]:
transposed = transposed.reindex(columns=['Date','ROUTER 1', 'TRANSCIEVER', 'SWITCH 1', 'ACCESS POINT 1',
       'ACCESS POINT 2', 'SWITCH 2', 'SWITCH 3', 'POWER SUPPLY 1', 'SWITCH 4',
       'SWITCH 5', 'SWITCH 6', 'ACCESS POINT 3', 'SUPERVISOR ENGINE',
       'SWITCH 7', 'WIRELESS CONTROLLER', 'SWITCH 8', 'SWITCH 9',
       'ACCESS POINT 4', 'SWITCH 10', 'POWER SUPPLY 2'])

In [259]:
transposed.to_csv('unimputed.csv')

assuming that missing values were actually missing and there were existing sales

In [260]:
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

new = transposed.copy()
new.drop(['Date'],axis=1, inplace=True)
mask = new.isna()
imputer = IterativeImputer(random_state=42)
imputed = imputer.fit_transform(new)
imputed_df = pd.DataFrame(imputed, index=new.index, columns=new.columns)
new[mask] = imputed_df[mask]
new

c:\Users\kedar\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\impute\_iterative.py:796: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


Product Name,ROUTER 1,TRANSCIEVER,SWITCH 1,ACCESS POINT 1,ACCESS POINT 2,SWITCH 2,SWITCH 3,POWER SUPPLY 1,SWITCH 4,SWITCH 5,SWITCH 6,ACCESS POINT 3,SUPERVISOR ENGINE,SWITCH 7,WIRELESS CONTROLLER,SWITCH 8,SWITCH 9,ACCESS POINT 4,SWITCH 10,POWER SUPPLY 2
0,1176.874240,87173.0,12.617558,35383.653311,48605.014593,13432.0,28585.0,115709.0,4253.699954,4571.0,5090.450248,65048.0,2173.430614,6073.0,1208.0,4.0,6187.0,28466.918282,24885.026733,28279.0
1,1203.880419,102849.0,6431.362819,34595.130943,62326.892737,12463.0,19686.0,98337.0,4316.469328,4100.0,5568.048415,67810.0,1798.500089,5459.0,963.0,-4.0,5301.0,34195.099511,20614.045510,22704.0
2,301.705754,126375.0,1651.992349,14749.609871,10265.570119,13452.0,44724.0,147681.0,3791.645150,4830.0,2655.987195,90928.0,2323.884165,6736.0,1698.0,36.0,7172.0,12525.940084,28536.875855,24098.0
3,906.000000,128629.0,3651.000000,37406.301862,-59390.459949,14581.0,37781.0,185327.0,177.000000,5472.0,2356.596557,148547.0,1692.827606,12213.0,1833.0,305.0,7582.0,-4167.431660,49038.728302,31461.0
4,941.000000,179125.0,10409.000000,38594.816720,-132651.395018,15831.0,31905.0,194170.0,270.000000,7391.0,1880.704828,228945.0,177.646793,13883.0,2423.0,1093.0,10964.0,-19098.241195,69943.094650,47009.0
5,663.000000,138750.0,10575.000000,57206.758834,-122373.227714,13864.0,37888.0,220386.0,850.000000,7894.0,1641.076598,204184.0,1169.219523,27045.0,2602.0,2360.0,11501.0,-20113.903755,65429.368701,35145.0
6,769.000000,208760.0,17934.000000,9576.000000,-162594.048404,15012.0,32689.0,204604.0,2717.000000,7101.0,222.000000,245453.0,15.000000,15162.0,2341.0,4317.0,14980.0,-28756.793924,77480.064520,45978.0
7,3565.000000,116126.0,15104.000000,46029.000000,-180301.038476,11349.0,30442.0,194538.0,3833.000000,6943.0,1549.000000,245803.0,108.000000,13855.0,1830.0,4343.0,9158.0,-23217.165384,101991.872239,34518.0
8,1020.000000,150803.0,15525.000000,109452.000000,26769.000000,10422.0,27028.0,185160.0,4279.000000,6126.0,4619.000000,136283.0,1972.000000,12351.0,1567.0,4094.0,11601.0,13260.000000,1227.000000,40605.0
9,1670.000000,82163.0,6938.000000,54955.000000,33670.000000,8726.0,21246.0,147284.0,6089.000000,4616.0,4764.000000,76020.0,2427.000000,8933.0,1210.0,2756.0,3788.0,21667.000000,24186.000000,26925.0


In [261]:
new.insert(0,'Date',transposed['Date'])


In [262]:
new.to_csv('iterative_imputed.csv')

assuming that missing values imply product didn't exist or distribution of product was not ongoing

In [263]:
new_0 = transposed.copy()
new_0.fillna(0, inplace=True)

In [264]:
new_0.to_csv('zero_imputated.csv')

In [267]:
! pip install fbprophet

     ---------------------------------------- 64.0/64.0 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 47.9/47.9 kB 2.5 MB/s eta 0:00:00
     ---------------------------------------- 5.8/5.8 MB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 91.8/91.8 kB 2.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of pystan to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of pystan to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for g

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [6 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\kedar\AppData\Local\Temp\pip-install-xr2g5yyu\pystan_ec232d0dded54d729a363a87c79252fb\setup.py", line 122, in <module>
          from Cython.Build.Inline import _get_build_extension
      ModuleNotFoundError: No module named 'Cython'
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
